### Init Environment

In [1]:
print('Installing torchprofile...')
!pip install torchprofile
print('All required packages have been successfully installed!')

Installing torchprofile...
All required packages have been successfully installed!


In [2]:
from collections import OrderedDict, defaultdict

import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
from torch.utils.mobile_optimizer import optimize_for_mobile


### Data

In [3]:
annotation_train_path = './dataset/annotation_train.csv'
dataset_train_path = './dataset/data_train.npy'
annotation_test_path = './dataset/annotation_test.csv'
dataset_test_path = './dataset/data_test.npy'

class RunningDataset(Dataset):
    def __init__(self,annotation_path, dataset_path, transform=None):
        self.transform = transform
        self.labels = pd.read_csv(annotation_path, header=None)
        self.data = np.load(dataset_path)

        print (self.data.shape)
        print (self.labels.shape)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = torch.tensor(sample, dtype=torch.float32)
        label = self.labels.iloc[idx,0]
        label = torch.tensor(label, dtype=torch.long)

        if self.transform:
            sample = self.transform(sample)
        
        return sample, label

train_dataset = RunningDataset(annotation_train_path, dataset_train_path)
test_dataset = RunningDataset(annotation_test_path, dataset_test_path)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=300,
    num_workers=0,
    pin_memory=True,
    shuffle=True,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=300,
    num_workers=0,
    pin_memory=True,
    shuffle=True,
)

(78422, 80, 6)
(78422, 1)
(16267, 80, 6)
(16267, 1)


### Model

In [4]:
class MyModel(nn.Module):
  def __init__(self, fc_size=196, hidden_size=42) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)
    self.seq_len = 80
    self.hidden_size = hidden_size
    self.fc_size = fc_size

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    self.gru1 = nn.GRU(input_size=6, hidden_size=self.hidden_size, batch_first=True)
    self.relu1 = nn.ReLU()
    self.gru2 = nn.GRU(input_size=self.hidden_size, hidden_size=self.hidden_size, batch_first=True)
    self.relu2 = nn.ReLU()
    add("conv", nn.Conv1d(self.hidden_size, 80, 2, stride=2))
    add("relu", nn.ReLU())
    add("pool", nn.MaxPool1d(4, padding=1))
    add("conv", nn.Conv1d(80, self.fc_size, 2, stride=1))
    add("relu", nn.ReLU())
    add("adaptivePoll",nn.AdaptiveAvgPool1d(1))
    add("bn", nn.BatchNorm1d(self.fc_size, eps=1e-06))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(self.fc_size, 4)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x, _ = self.gru1(x)
    x = self.relu1(x)
    x, _ = self.gru2(x)
    x = self.relu2(x)
    x = x.reshape(-1, self.hidden_size, self.seq_len)
    x = self.backbone(x)
    x = x.reshape(-1, self.fc_size)
    x = self.classifier(x)
    x = self.softmax(x)

    return x
  
model = MyModel().cuda()
## show model summary
print(model.eval())

MyModel(
  (gru1): GRU(6, 42, batch_first=True)
  (relu1): ReLU()
  (gru2): GRU(42, 42, batch_first=True)
  (relu2): ReLU()
  (backbone): Sequential(
    (conv0): Conv1d(42, 80, kernel_size=(2,), stride=(2,))
    (relu0): ReLU()
    (pool0): MaxPool1d(kernel_size=4, stride=4, padding=1, dilation=1, ceil_mode=False)
    (conv1): Conv1d(80, 196, kernel_size=(2,), stride=(1,))
    (relu1): ReLU()
    (adaptivePoll0): AdaptiveAvgPool1d(output_size=1)
    (bn0): BatchNorm1d(196, eps=1e-06, momentum=0.1, affine=True, track_running_stats=True)
  )
  (classifier): Linear(in_features=196, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


### Training

In [5]:
def train(
  model: nn.Module,
  dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  callbacks = None
) -> None:
  model.train()

  for inputs, targets in tqdm(dataloader, desc='train', leave=False):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Reset the gradients (from the last iteration)
    optimizer.zero_grad()

    # Forward inference
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward propagation
    loss.backward()

    # Update optimizer and LR scheduler
    optimizer.step()
    scheduler.step()

    if callbacks is not None:
        for callback in callbacks:
            callback()

@torch.inference_mode()
def evaluate(
  model: nn.Module,
  dataloader: DataLoader,
  verbose=True,
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0

  for inputs, targets in tqdm(dataloader, desc="eval", leave=False,
                              disable=not verbose):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Inference
    outputs = model(inputs)
   
    outputs = outputs.argmax(dim=1)

    # Update metrics
    num_samples += targets.size(0)
    # abnormal = 0, normal = 3
    outputs[outputs < 3] = 0
    targets[targets < 3] = 0
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

# save checkpoint to local file
def save_checkpoint(
  model: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  epoch: int,
  accuracy: float,
  path: str,
) -> None:
  torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'scheduler': scheduler.state_dict(),
    'epoch': epoch,
    'accuracy': accuracy,
  }, f"{path}")


# load checkpoint from local file
def load_checkpoint(
  model: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  path: str,
) -> None:
  checkpoint = torch.load(path)

  model.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  scheduler.load_state_dict(checkpoint['scheduler'])

  return checkpoint['epoch'], checkpoint['accuracy']

# train model
def run(
  model: nn.Module,
  train_dataloader: DataLoader,
  valid_dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  num_epochs: int,
  checkpoint_path: str,
  verbose=True,
) -> None:
  best_accuracy = 0.0

  for epoch in tqdm(range(num_epochs)):
    # Train for one epoch
    train(model, train_dataloader, criterion, optimizer, scheduler)

    # Evaluate on validation set
    accuracy = evaluate(model, valid_dataloader, verbose)
    # Save checkpoint
    if accuracy > best_accuracy:
      best_accuracy = accuracy
      save_checkpoint(model, optimizer, scheduler, epoch, accuracy,
                      checkpoint_path)

    if verbose:
      print(f'Epoch: {epoch + 1:03d} | '
            f'accuracy = {accuracy:.2f} |'
            f'best_accuracy = {best_accuracy:.2f}')

In [11]:
torch.autograd.set_detect_anomaly(True)

num_epochs =15
steps_per_epoch = len(train_dataloader)

def run_per_model(model,lr=2e-3,gamma=0.5):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=steps_per_epoch, gamma=gamma)
    criterion = nn.CrossEntropyLoss()

    checkpoint_path = "checkpoint.pth"
    run(model, train_dataloader, test_dataloader, criterion, optimizer, scheduler, num_epochs, checkpoint_path, verbose=True)


run_per_model(model)


  0%|          | 0/15 [00:00<?, ?it/s]

train:   0%|          | 0/262 [00:00<?, ?it/s]

eval:   0%|          | 0/55 [00:00<?, ?it/s]

Epoch: 001 | accuracy = 78.32 |best_accuracy = 78.32


train:   0%|          | 0/262 [00:00<?, ?it/s]

...

eval:   0%|          | 0/55 [00:00<?, ?it/s]

Epoch: 014 | accuracy = 77.72 |best_accuracy = 79.39


train:   0%|          | 0/262 [00:00<?, ?it/s]

eval:   0%|          | 0/55 [00:00<?, ?it/s]

Epoch: 015 | accuracy = 77.55 |best_accuracy = 79.39


In [9]:
# import model
model = MyModel().cuda()
# load checkpoint
steps_per_epoch = len(train_dataloader)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=steps_per_epoch, gamma=0.5)
criterion = nn.CrossEntropyLoss()
load_checkpoint(model, optimizer, scheduler, "checkpoint.pth")
# check model accuracy
acc = evaluate(model, test_dataloader, verbose=True)
print("Accuracy of the model: ", acc)


model.eval()  # Set the model to evaluation mode

example = torch.rand(1, 80, 6).cuda()

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)
# save
traced_script_module.save("model.pt")

optimized_torchscript_model = optimize_for_mobile(traced_script_module)

# Save the TorchScript model
optimized_torchscript_model._save_for_lite_interpreter("optimized_model.ptl")


# compare model size

basic_model_size = os.path.getsize("model.pt")
print("basic model size: ", basic_model_size)
optimized_model_size = os.path.getsize("optimized_model.pt")
print("optimized model size: ", optimized_model_size)


eval:   0%|          | 0/55 [00:00<?, ?it/s]

Accuracy of the model:  79.3877182006836
basic model size:  260638
optimized model size:  191773
